<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 1 : Cinétique des gaz parfaits</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

Antoine Veillette

# Introduction #
Ce travail révise d'abord quelques bases générales de mécanique statistique classique avec le script `tds2Danimation_hXX.py` qui simule la théorie cinétique des gaz parfaits en 2D. Cette simulation utilise le concept de sphères dures, mais ici pour le cas des particules d'un gaz afin d'introduire des collisions élastiques entre elles sur leurs trajectoires ballistiques. Notez qu'une sphère est colorée et grossie seulement pour l’effet visuel dans l'animation, la physique de l’algorithme codé considère bien des particules totalement identiques. Les questions sur cette simulation, à répondre directement dans les cellules du carnet _(Notebook)_ ici-même, explorent quelques paramètres de la thermodynamique statistique et introduisent de nouveaux termes utiles à l'étude du mouvement des électrons dans la matière.

_N.B._ 
- _Pour montrer les animations à l'écran, le script `tds2Danimation_hXX.py` importe la librairie `VPython` qu'il faut donc installer. Des liens vers sa documentation et de l'information complémentaire sont donnés dans la médiagraphie à la fin._
- _Le code dans ce script est abusivement commenté dans notre contexte pédagogique, mais il serait bien sûr préférable de s’en tenir aux recommandations du <a href="https://www.python.org/dev/peps/pep-0008"> PEP 8 — Style Guide for Python Code</a>._
- _Notez finalement que la boucle principale à la fin du script laisse l'utilisateur voir l'animation aussi longtemps que souhaité, assurez-vous donc de savoir comment l'interrompre correctement avant de lancer la simulation ou de la remplacer par une boucle `for`._

# 1<sup>re</sup> partie - Cinétique CLASSIQUE des gaz parfaits #

### Simulation 2D ###

In [1]:
#!pip install matplotlib
%run tds2Danimation_h25.py
# ou
#%run tds3Dsim_h25.py #si disponible : script sans la lente animation Vpython qui est en développement à la session h25!
# Remplacez "XX" par les deux derniers chiffres de l'année de votre session.
# N'hésitez pas à exécuter l'animation 2D ou la simulation 3D à l'extérieur du _Notebook_. Cette cellule vise à préciser que les questions qui suivent se basent sur ces scripts et doivent mener aux mêmes répomses autant en 2D qu'en 3D.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

### Questions statistiques ###

**I.** _(3 points)_  &mdash; Utilisez la liste finale des vecteurs de quantité de mouvement $\vec{p}$ de toutes les sphères pour calculer la moyenne de son carré $\langle p^2\rangle=\langle\vec{p}\cdot\vec{p}\rangle$ en écrivant le code nécessaire dans la cellule qui suit. 

In [2]:
psq = np.array([dot(i,i) for i in p])
N = len(psq)
average_psquared = psq.sum()/N
average_psquared

np.float64(5.599999999999997e-47)


**II.** _(2 points)_  &mdash; La température $T$ (macroscopique) est proportionnelle à l'énergie cinétique moyenne $E_{cin}$ de l'ensemble des particules lorsque ce système est rendu à l'équilibre. Celle-ci peut se calculer classiquement selon son <a href="https://fr.wikipedia.org/wiki/%C3%89quipartition_de_l%27%C3%A9nergie">principe d'équipartition</a>, _i.e._ en répartissant l'énergie également sur chaque degré de liberté ici en translation seulement d'où, au total pour $\text{DIM}=1,2\text{ ou } 3$ dimensions d'espace réel,
\begin{equation}
E_{cin}=\frac{\langle p^2 \rangle}{2m}=\text{DIM}\times\frac{1}{2}k_BT
\end{equation}
avec $k_B$, la constante de Boltzmann et $m$, la masse de chaque particule. Quelle est la température du gaz de sphères dures à la fin de la simulation? Est-ce qu'elle a changé significativement par rapport à sa valeur initiale?

In [3]:
average_psquared / (2*mass*k)

np.float64(299.9999999999999)

In [4]:
E_cin = average_psquared / (2*mass)
T_est = 2* E_cin/(DIM*k)
delta_T = abs(T - T_est)

In [5]:
%whos
#

Variable              Type                          Data/Info
-------------------------------------------------------------
Atoms                 list                          n=200
Camera                type                          <class 'vpython.vpython.Camera'>
DIM                   int                           2
E_cin                 float64                       4.199999999999998e-21
GSversion             list                          n=2
GlowWidget            type                          <class 'vpython.vpython.GlowWidget'>
L                     int                           1
Mouse                 type                          <class 'vpython.vpython.Mouse'>
N                     int                           200
Natoms                int                           200
RackOutline           function                      <function RackOutline at 0x10676c360>
Ratom                 float                         0.01
T                     int                           300
T_est  

**III.** _(10 points)_ &mdash; Modifiez le code de la simulation pour ajouter une fonction qui suit la trajectoire d'UNE SEULE particule, c'est-à-dire qu'elle doit enregistrer, dans une liste, des valeurs de variables pour cette particule et ce, à chacune de ses collisions avec une autre particule (_i.e._ excluez les collisions avec les parois de la boîte). Les deux variables scalaires à lister sont:
- la distance que la particule a parcouru entre chaque collision,
- le temps écoulé entre ces collisions.

Copiez le code de votre fonction dans la cellule qui suit en y commentant clairement les variables pour ces listes qui devront persister après avoir interrompu l'exécution de la simulation. N'oubliez pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

In [12]:
dist = np.array(tracked_data.distances)
times = np.array(tracked_data.times)
times


array([6.85981106e-06, 7.66144948e-06, 1.39843238e-05, 1.22661279e-05,
       9.00036341e-06, 1.02432377e-05, 1.30766284e-05, 6.26045047e-06,
       1.24393435e-05, 4.57862892e-06, 1.73141090e-05, 6.72049525e-06,
       1.58051334e-05, 1.96889849e-05, 1.62001400e-05, 1.01494556e-05,
       1.18319407e-05, 1.09834306e-05, 9.81388027e-06, 1.31728812e-05,
       1.39790314e-05, 7.80340010e-06, 9.63155923e-06, 1.01008153e-05,
       1.06671415e-05, 1.60915455e-05, 9.57571094e-06, 1.17571147e-05,
       2.40103511e-05, 3.40103511e-05, 7.23429725e-06, 1.08674476e-05,
       4.68568941e-06, 1.34936440e-05, 1.43407434e-05, 5.70555117e-06,
       1.36492248e-05, 7.03198311e-06, 1.25813090e-05, 6.89261192e-06,
       7.92353014e-06, 1.42053730e-05, 8.46058202e-06, 2.09837559e-05,
       3.29985617e-05, 2.09407123e-05, 3.09407123e-05, 7.49461448e-06,
       5.74294701e-06, 1.61669653e-05, 9.73167603e-06, 5.91827986e-06,
       7.79817872e-06, 1.69537196e-05, 1.65216736e-05, 1.28429706e-05,
      

**IV.** _(2 points)_ &mdash; Calculez le **libre parcours moyen** $l_{moy}$ et le **temps de collision** $\tau$ qui sont les valeurs moyennes des deux listes compilées au numéro précédent.

_(Pour votre information, le libre parcours moyen est de l’ordre de 100 nm dans l'air à température et pression ambiantes, mais_ $l_{moy}$ _peut dépasser 100 000 km dans une enceinte sous vide avec les technologies de pompes modernes!)_



In [26]:
libre_parcous_moy = dist.sum()/len(dist)
temps_col_tau = times.sum()/len(times)
times

array([6.85981106e-06, 7.66144948e-06, 1.39843238e-05, 1.22661279e-05,
       9.00036341e-06, 1.02432377e-05, 1.30766284e-05, 6.26045047e-06,
       1.24393435e-05, 4.57862892e-06, 1.73141090e-05, 6.72049525e-06,
       1.58051334e-05, 1.96889849e-05, 1.62001400e-05, 1.01494556e-05,
       1.18319407e-05, 1.09834306e-05, 9.81388027e-06, 1.31728812e-05,
       1.39790314e-05, 7.80340010e-06, 9.63155923e-06, 1.01008153e-05,
       1.06671415e-05, 1.60915455e-05, 9.57571094e-06, 1.17571147e-05,
       2.40103511e-05, 3.40103511e-05, 7.23429725e-06, 1.08674476e-05,
       4.68568941e-06, 1.34936440e-05, 1.43407434e-05, 5.70555117e-06,
       1.36492248e-05, 7.03198311e-06, 1.25813090e-05, 6.89261192e-06,
       7.92353014e-06, 1.42053730e-05, 8.46058202e-06, 2.09837559e-05,
       3.29985617e-05, 2.09407123e-05, 3.09407123e-05, 7.49461448e-06,
       5.74294701e-06, 1.61669653e-05, 9.73167603e-06, 5.91827986e-06,
       7.79817872e-06, 1.69537196e-05, 1.65216736e-05, 1.28429706e-05,
      

**V.** _(2 points)_ Calculez la vitesse $\vec{v}$ de la particule entre chaque paire de collisions. Quelle est la vitesse moyenne $\langle\vec{v}\rangle$ de la particule?

In [30]:
v = dist/times
moy_v = v.sum()/v.size

**VI.** _(5 points)_ &mdash; Pour cette même liste de vitesses, comparez les distributions de la norme $||\vec{v}||$, du carré $v^2$ et d’une de ses composantes $v_x^2$ en étalonnant l’abscisse pour contraster les histogrammes avec une échelle appropriée. Indiquez sur ce graphique la moyenne, le mode, la médiane et la moyenne quadratique des distributions.

In [35]:
norm = np.abs(v)
v_sq = v**2
v[1]
#

np.float64(2368.3683271667583)

**Bonus.** _(4 points)_ &mdash; Montrez que 
- (a) le théorème central limite est satisfait par une des distributions de vitesse du numéro précédent,
- (b) le système simulé est ergodique.

In [ ]:

#

# Médiagraphie #
 - La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour faciliter la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le code adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.